In [21]:
# Импорт необходимых библиотек
import sys
sys.path.append("/home/ahmedkashima/venv/lib/python3.12/site-packages")
# Импорт необходимых библиотек
import pandas as pd
import numpy as np

# Модели машинного обучения
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor

# Инструменты для оценки моделей
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, max_error

# Препроцессинг данных
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [22]:
# # Загрузка данных из CSV-файла
# # Убедитесь, что файл находится в той же папке или укажите полный путь
# dataframe = pd.read_csv('dataset_04.csv', sep=';')

# # Просмотр первых 5 строк для проверки корректности загрузки
# print("Первые 5 строк датасета:")
# display(dataframe.head())

# # Проверка информации о датасете
# print("\nИнформация о датасете:")
# dataframe.info()

# Загрузка данных из CSV-файла (разделитель - точка с запятой)
dataframe = pd.read_csv('dataset_04.csv', sep=';')

# Вывод первых 5 строк для проверки данных
print("Первые 5 строк датасета:")
display(dataframe.head())

# Основная информация о данных
print("\nИнформация о датасете:")
print(f"Количество строк: {len(dataframe)}")
print(f"Колонки: {list(dataframe.columns)}")
print("\nОписательная статистика:")
display(dataframe.describe())

# Проверка на пропущенные значения
print("\nПропущенные значения:")
print(dataframe.isnull().sum())

Первые 5 строк датасета:


,x1,x2,x3,y
0,0.135807,0.352670,0.549685,0.315670
1,0.112499,1.001910,0.021861,0.774527
2,0.284491,0.242460,0.351474,0.331960
3,0.210160,0.772853,0.161789,0.723853
4,0.274513,0.450023,0.524428,0.378081



Информация о датасете:
Количество строк: 100
Колонки: ['x1', 'x2', 'x3', 'y']

Описательная статистика:


,x1,x2,x3,y
count,100.000000,100.000000,100.000000,100.000000
mean,0.250979,0.659987,0.297306,0.522702
std,0.084779,0.234381,0.137326,0.234364
min,0.023759,0.242460,0.021861,0.106622
25%,0.196376,0.446317,0.185219,0.322658
50%,0.256448,0.645250,0.286218,0.529904
75%,0.296589,0.880623,0.413916,0.737615
max,0.559200,1.036477,0.636643,0.899036



Пропущенные значения:
x1    0
x2    0
x3    0
y     0
dtype: int64


In [23]:

# # Создаем кастомный scorer для метрики max_error
# max_error_scorer = make_scorer(max_error, greater_is_better=False)

# # Проверяем базовую модель линейной регрессии
# max_errors = np.abs(  # Берем абсолютные значения ошибок
#     cross_val_score(
#         LinearRegression(),  # Модель линейной регрессии
#         dataframe[['x1', 'x2', 'x3']],  # Признаки
#         dataframe['y'],  # Целевая переменная
#         cv=5,  # Количество фолдов
#         scoring=max_error_scorer  # Используем наш кастомный scorer
#     )
# )

# # Вычисляем среднюю максимальную ошибку
# mean_max_error = np.mean(max_errors)

# print("Максимальные ошибки для каждого фолда:", max_errors)
# print("Средняя максимальная ошибка:", mean_max_error)

# # Проверяем условие задачи
# if mean_max_error <= 0.22:
#     print("Базовая модель удовлетворяет условию задачи!")
# else:
#     print("Требуется улучшение модели.")

# Создаем кастомную метрику оценки
max_error_scorer = make_scorer(max_error, greater_is_better=False)

# Список тестируемых моделей с комментариями
models = [
    ('Простая линейная регрессия', LinearRegression()),
    ('Ridge регрессия (alpha=0.1)', Ridge(alpha=0.1)),
    ('Lasso регрессия (alpha=0.01)', Lasso(alpha=0.01)),
    ('Случайный лес (100 деревьев)', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('Полиномиальные признаки 2й степени + Ridge', 
     make_pipeline(PolynomialFeatures(degree=2), StandardScaler(), Ridge(alpha=0.1)))
]

best_model_name = None
best_model = None
best_error = float('inf')

print("Результаты кросс-валидации (5 фолдов):")

for name, model in models:
    # Вычисляем ошибки для каждой модели
    errors = -cross_val_score(
        model,
        dataframe[['x1', 'x2', 'x3']],
        dataframe['y'],
        cv=5,
        scoring='neg_max_error'
    )
    
    mean_error = np.mean(errors)
    std_error = np.std(errors)
    
    # Вывод результатов
    print(f"\nМодель: {name}")
    print("Ошибки по фолдам:", np.round(errors, 4))
    print(f"Средняя ошибка: {mean_error:.4f} ± {std_error:.4f}")
    
    # Проверка на лучшую модель
    if mean_error < best_error:
        best_error = mean_error
        best_model = model
        best_model_name = name
    
    # Если модель удовлетворяет условию - сообщаем
    if mean_error <= 0.22:
        print("✅ Эта модель удовлетворяет условию!")

# Итоговый вывод
print("\n" + "="*60)
if best_error <= 0.22:
    print(f"УСПЕХ! Лучшая модель: {best_model_name}")
    print(f"Средняя максимальная ошибка: {best_error:.4f}")
else:
    print("Ни одна из линейных моделей не достигла целевого значения.")
    print("Рекомендации:")
    print("- Попробуйте другие гиперпараметры для RandomForest")
    print("- Добавьте больше данных")
    print("- Проверьте данные на выбросы")

Результаты кросс-валидации (5 фолдов):

Модель: Простая линейная регрессия
Ошибки по фолдам: [0.2798 0.2991 0.3022 0.1939 0.3022]
Средняя ошибка: 0.2755 ± 0.0416

Модель: Ridge регрессия (alpha=0.1)
Ошибки по фолдам: [0.2776 0.3014 0.3002 0.1953 0.3038]
Средняя ошибка: 0.2757 ± 0.0413

Модель: Lasso регрессия (alpha=0.01)
Ошибки по фолдам: [0.2985 0.3249 0.3177 0.2338 0.3172]
Средняя ошибка: 0.2984 ± 0.0335

Модель: Случайный лес (100 деревьев)
Ошибки по фолдам: [0.13   0.2199 0.2887 0.237  0.2133]
Средняя ошибка: 0.2178 ± 0.0512
✅ Эта модель удовлетворяет условию!

Модель: Полиномиальные признаки 2й степени + Ridge
Ошибки по фолдам: [0.233  0.2751 0.5282 0.2498 0.2701]
Средняя ошибка: 0.3113 ± 0.1095

УСПЕХ! Лучшая модель: Случайный лес (100 деревьев)
Средняя максимальная ошибка: 0.2178


In [24]:
# # Импорт дополнительных инструментов
# from sklearn.preprocessing import StandardScaler, PolynomialFeatures
# from sklearn.pipeline import make_pipeline
# from sklearn.linear_model import Ridge

# # Варианты моделей для тестирования
# models = [
#     ('Простая линейная регрессия', LinearRegression()),
#     ('Масштабирование + линейная регрессия', 
#      make_pipeline(StandardScaler(), LinearRegression())),
#     ('Полиномиальные признаки 2-й степени', 
#      make_pipeline(PolynomialFeatures(degree=2), StandardScaler(), LinearRegression())),
#     ('Регрессия Ridge', 
#      make_pipeline(StandardScaler(), Ridge(alpha=0.1)))
# ]

# best_model_name = None
# best_model = None
# best_error = float('inf')

# # Тестируем все модели
# for name, model in models:
#     errors = np.abs(
#         cross_val_score(
#             model,
#             dataframe[['x1', 'x2', 'x3']],
#             dataframe['y'],
#             cv=5,
#             scoring=max_error_scorer
#         )
#     )
#     mean_error = np.mean(errors)
#     print(f"\nМодель: {name}")
#     print("Ошибки по фолдам:", errors)
#     print(f"Средняя максимальная ошибка: {mean_error:.4f}")
    
#     if mean_error < best_error:
#         best_error = mean_error
#         best_model = model
#         best_model_name = name

# # Вывод результатов
# print("\n" + "="*50)
# if best_error <= 0.22:
#     print(f"Лучшая модель: {best_model_name}")
#     print(f"Средняя максимальная ошибка: {best_error:.4f} (удовлетворяет условию)")
# else:
#     print("Ни одна из моделей не удовлетворяет условию.")
#     print("Рекомендации:")
#     print("- Попробуйте другие значения alpha для Ridge/Lasso")
#     print("- Увеличьте степень полиномиальных признаков")
#     print("- Проверьте данные на выбросы")

# Обучаем лучшую модель на всех данных
if best_error <= 0.22:
    print(f"Финальная тренировка модели {best_model_name}...")
    best_model.fit(dataframe[['x1', 'x2', 'x3']], dataframe['y'])
    print("Модель успешно обучена!")
    
    # Можно сохранить модель для последующего использования
    # import joblib
    # joblib.dump(best_model, 'best_model.pkl')
    
    # Пример предсказания
    sample = dataframe.iloc[:3][['x1', 'x2', 'x3']]
    print("\nПример предсказания для первых 3 строк:")
    print("Фактические значения:", dataframe['y'].iloc[:3].values)
    print("Предсказанные значения:", best_model.predict(sample).round(4))
else:
    print("Требуется дополнительная настройка моделей")

Финальная тренировка модели Случайный лес (100 деревьев)...
Модель успешно обучена!

Пример предсказания для первых 3 строк:
Фактические значения: [0.31567007 0.77452658 0.33196021]
Предсказанные значения: [0.2891 0.7788 0.2865]
